### main.ipynb – Datenextraktion & Vorverarbeitung
In diesem Notebook wird die strukturierte Verarbeitung von Online-Medienbeiträgen vorbereitet.  
Die wichtigsten Schritte:
- `.tar.gz`-Dateien entpacken (HTML-Dateien extrahieren)
- HTML-Inhalte bereinigen und Stoppwörter entfernen
- Wortfrequenzen je Medium und Datum zählen
- Ergebnisse in einer SQLite-Datenbank (`output/dwh.sqlite3`) speichern




##### 1. Import der benötigten Pakete

In [1]:
# Standard
import os # Dateipfaden
import pandas as pd # Tabellenverarbeitung (DataFrames)

# Einlesen & entpacken Archivdateien
from glob import glob # Mehrere Dateien suchen
import tarfile # .tar.gz Dateien entpacken

# Bearbeiten von html-Dateien
from bs4 import BeautifulSoup  # HTML auslesen und bereinigen
import requests # HTTP-Anfragen

# Speicherung
from datetime import datetime # Datumsangaben
import sqlite3 # SQL-Datenbank

In [2]:
# Pfade 

# Projektverzeichnis (z. B. .../notebooks)
PROJECT_ROOT = os.getcwd()
print(PROJECT_ROOT)

# Input-Pfade
INPUT_PATH = os.path.join(PROJECT_ROOT, "..", "input", "raw")
DATA_LAKE_PATH = os.path.join(INPUT_PATH, "data-lake")         # HTML- und CSV-Dateien
ZIP_PATH = os.path.join(INPUT_PATH, "downloaded_zips")         # ZIP-Dateien

# Output-Pfade
OUTPUT_PATH = os.path.join(PROJECT_ROOT, "..", "output")
STORAGE_PATH = os.path.join("..", "input", "raw", "data-lake")
SQL_PATH = os.path.join(OUTPUT_PATH, "dwh.sqlite3")               # SQLite-Datenbank
CSV_PATH = os.path.join(OUTPUT_PATH, "wordcount_news.csv")        # Exportierte CSV


# Stoppwörter einmalig laden
STOPWORDS_URL = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list = requests.get(STOPWORDS_URL, allow_redirects=True).text.split("\n")[9:]

d:\DBU\ADSC11 ADS-01\Studienarbeit\newspaper-scraping\notebooks


#### 2. Archiv entpacken und Dateien speichern

In [ ]:
# Funktion: Mit tarfile Rohdaten (Format .tar.gz) einlesen, extrahieren & entpacken
def extract_tar_file(ZIP_PATH, DATA_LAKE_PATH): 
    with tarfile.open(ZIP_PATH, "r:gz") as tar:
        for member in tar.getmembers():
            # Zielpfad für jede Datei 
            filename = os.path.basename(member.name)
            
            # HTML- und CSV-Dateien extrahieren
            if filename.endswith(".html") or filename.endswith(".csv"):
                target_path = os.path.join(DATA_LAKE_PATH, filename)
                
                # Skippen, wenn Datei schon existiert
                if os.path.exists(target_path):
                    continue
                
                # Entpacken in HTML-Datei
                with open(target_path, "wb") as f:
                    f.write(tar.extractfile(member).read())
                
                print(f"Entpackt: {filename}")

In [5]:
# .tar.gz-Dateien im ZIP-Ordner durchlaufen
print("[INFO] Entpackung gestartet...")
for zip_file in os.listdir(ZIP_PATH):
    if zip_file.endswith(".tar.gz"):
        zip_path = os.path.join(ZIP_PATH, zip_file)
        print(f"[INFO] Entpacke Archiv: {zip_path}")
        # Für jede Datei die Funktion extract_tar_file() aufrufen
        extract_tar_file(zip_path, DATA_LAKE_PATH)

[INFO] Entpackung gestartet...
[INFO] Entpacke Archiv: c:\Users\Marta\Documents\Marta\DBU\ADSC11 ADS-01\Studienarbeit\newspaper-scraping\notebooks\..\input\raw\downloaded_zips\2021-04.tar.gz


KeyboardInterrupt: 

In [9]:
# Check, ob CSV- & HTML-Dateien geladen wurden
# Anzahl HTML- und CSV-Dateien im data-lake
html_files = [f for f in os.listdir(DATA_LAKE_PATH) if f.endswith(".html")]
csv_files = [f for f in os.listdir(DATA_LAKE_PATH) if f.endswith(".csv") and not f.startswith("~")]

print("Anzahl CSV-Dateien:", len(csv_files))
print("Anzahl HTML-Dateien:", len(html_files))

Anzahl CSV-Dateien: 1429
Anzahl HTML-Dateien: 79970


#### 3. Funktionen definieren

In [ ]:
# Funktion: HTML-Datei mit Eincoding einlesen
def read_html_file(filepath, encoding="utf-8"):
    with open(filepath, "r", encoding=encoding) as f:
        return f.read()

In [ ]:
# Funktion: HTML-Text bereinigen & in einzelne Wörter zerlegen
def process_html(html):
    # HTML-Parser, nur Text auslesen
    bstext = BeautifulSoup(html, "html.parser")
    text = bstext.get_text(separator=" ").lower()

    # Zeilenumbrüche durch Leerzeichen ersetzen & Wörter trennen
    items = text.replace("\n", " ").split(" ")

    # Kurze Wörter und Stoppwörter entfernen
    items = [i for i in items if len(i) > 1 and i not in stopwords_list]
    return items

In [ ]:
# Funktion: HTML und Metadaten zusammenführen und Wörter zählen 
def process_newspaper(newspaper):
    filename = os.path.basename(newspaper["file_name"])  
    full_path = os.path.join(DATA_LAKE_PATH, filename)   

    # Encoding einlesen
    encoding = newspaper["encoding"].lower()
    
    # HTML-Datei laden & bereinigen
    # Funktion read_html_file aufrufen
    html = read_html_file(full_path, encoding)
    # Funktion process_html aufrufen
    items = process_html(html)

   # Häufigkeit jedes Wortes zählen
    count = pd.Series(items).value_counts()

    # Als DataFrame formatieren mit Spalten Wort, Medium, Datum
    count_df = count.to_frame()
    count_df.columns = ["count"]
    count_df["word"] = count_df.index
    count_df["source"] = newspaper["name"]
    count_df["date"] = newspaper["date"]

    return count_df

In [ ]:
# Funktion: Funktion process_newspaper anwenden, in der Variable collection speichern und Erfolg oder Fehler ausgeben
def process_wrapper(newspaper):
    name = newspaper["name"]
    try:
        count = process_newspaper(newspaper)       
        print(f"[INFO] Verarbeitung erfolgreich: {name}")
        collection.append(count)
    except Exception as e:
        print(f"[ERROR] Fehler bei {name}: {e}")

#### 4. Iteratives Anwenden der Verarbeitungsfunktionen auf die Medien

In [ ]:
# Zielmedien definieren: Cluster öffentlich-rechtlich
zielmedien_or = {"dlf", "tagesschau"}

In [ ]:
# 1. Import & Sammlung vorbereiten
collection = []

# 2. Ziel-CSV vorbereiten (falls vorhanden: löschen)
OUTPUT_CSV = "output/cluster_oeffentlich.csv"
if os.path.exists(OUTPUT_CSV):
    os.remove(OUTPUT_CSV)

# 3. Verarbeitung ausgewählter Dateien
for file in sorted(os.listdir(STORAGE_PATH)):
    if file.endswith(".csv"):
        filepath = os.path.join(STORAGE_PATH, file)
        try:
            df = pd.read_csv(filepath)

            # Filter: Nur Zielmedien verarbeiten
            df = df[df["name"].isin(zielmedien_or)]

            if not df.empty:
                print(f"[INFO] Datei: {file}")
                # Funktion process_wrapper aufrufen
                df.apply(process_wrapper, axis=1)

                # Zwischenergebnisse chunkweise speichern
                for chunk in collection:
                    chunk.to_csv(OUTPUT_CSV, mode="a", index=False, header=not os.path.exists(OUTPUT_CSV))
                collection = [] # RAM freigeben

        except Exception as e:
            print(f"[ERROR] Fehler bei Datei {file}: {e}")

[INFO] Datei: 2021-04-01.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Datei: 2021-04-02.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Datei: 2021-04-03.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Datei: 2021-04-04.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Datei: 2021-04-05.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Datei: 2021-04-06.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Datei: 2021-04-07.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Datei: 2021-04-08.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Datei: 2021-04-09.csv
[INFO] Verarbeitung erfolgreich: dlf
[INFO] Verarbeitung erfolgreich: tagesschau
[INFO] Dat

In [23]:
# Pfad zur gespeicherten CSV-Datei
csv_path = "output/cluster_oeffentlich.csv"

try:
    # CSV laden
    df = pd.read_csv(csv_path)

    # Anzahl Zeilen und Spalten anzeigen
    print("[INFO] Form der Datei:", df.shape)

    # Zufällige Beispielzeilen anzeigen
    print("[INFO] Beispielhafte Einträge:")
    print(df.sample(3, random_state=1))

except Exception as e:
    print("[ERROR] Fehler beim Laden der CSV-Datei:", e)

[INFO] Form der Datei: (3383792, 4)
[INFO] Beispielhafte Einträge:
         count       word source        date
2243207      2      jahre    dlf  2024-01-08
1060541      2     social    dlf  2022-10-13
2272109      1  finanzen,    dlf  2024-01-19


In [9]:
# Zielmedien: Wirtschaftsmedien
zielmedien_wm = {"handelsblatt", "wiwo", "mm", "pioneer", "boerse"}

In [ ]:
# 1. Import & Sammlung vorbereiten
collection = []

# 2. CSV-Datei vorbereiten
OUTPUT_CSV = "output/cluster_wirtschaft.csv"
if os.path.exists(OUTPUT_CSV):
    os.remove(OUTPUT_CSV)

# 3. Verarbeitung und direkt speichern (chunkweise!)
for file in sorted(os.listdir(STORAGE_PATH)):
    if file.endswith(".csv"):
        filepath = os.path.join(STORAGE_PATH, file)
        try:
            df = pd.read_csv(filepath)

            # Filter Zielmedien
            df = df[df["name"].isin(zielmedien_wm)]

            if not df.empty:
                print(f"[INFO] Datei: {file}")
                df.apply(process_wrapper, axis=1)

                # Sofort speichern
                for chunk in collection:
                    chunk.to_csv(OUTPUT_CSV, mode="a", index=False, header=not os.path.exists(OUTPUT_CSV))
                collection = []

        except Exception as e:
            print(f"[ERROR] Fehler bei Datei {file}: {e}")

[INFO] Datei: 2021-04-01.csv
[INFO] Verarbeitung erfolgreich: handelsblatt
[INFO] Verarbeitung erfolgreich: pioneer
[INFO] Verarbeitung erfolgreich: boerse
[INFO] Verarbeitung erfolgreich: mm
[INFO] Verarbeitung erfolgreich: wiwo
[INFO] Datei: 2021-04-02.csv
[INFO] Verarbeitung erfolgreich: handelsblatt
[INFO] Verarbeitung erfolgreich: pioneer
[INFO] Verarbeitung erfolgreich: boerse
[INFO] Verarbeitung erfolgreich: mm
[INFO] Verarbeitung erfolgreich: wiwo
[INFO] Datei: 2021-04-03.csv
[INFO] Verarbeitung erfolgreich: handelsblatt
[INFO] Verarbeitung erfolgreich: pioneer
[INFO] Verarbeitung erfolgreich: boerse
[INFO] Verarbeitung erfolgreich: mm
[INFO] Verarbeitung erfolgreich: wiwo
[INFO] Datei: 2021-04-04.csv
[INFO] Verarbeitung erfolgreich: handelsblatt
[INFO] Verarbeitung erfolgreich: pioneer
[INFO] Verarbeitung erfolgreich: boerse
[INFO] Verarbeitung erfolgreich: mm
[INFO] Verarbeitung erfolgreich: wiwo
[INFO] Datei: 2021-04-05.csv
[INFO] Verarbeitung erfolgreich: handelsblatt
[INFO

In [11]:
# Pfad zur gespeicherten CSV-Datei
csv_path = "output/cluster_wirtschaft.csv"

try:
    # CSV laden
    df = pd.read_csv(csv_path)

    # Anzahl Zeilen und Spalten anzeigen
    print("[INFO] Form der Datei:", df.shape)

    # Zufällige Beispielzeilen anzeigen
    print("[INFO] Beispielhafte Einträge:")
    print(df.sample(3, random_state=1))

except Exception as e:
    print("[ERROR] Fehler beim Laden der CSV-Datei:", e)

[INFO] Form der Datei: (8388888, 4)
[INFO] Beispielhafte Einträge:
         count                  word  source        date
387905       1                  uns,  boerse  2021-05-31
3355691      1         positionierte    wiwo  2022-09-06
3614777      1  energieinfrastruktur    wiwo  2022-10-17


In [13]:
# 1. Sammlung vorbereiten
collection = []

# 2. Zielmedien definieren
zielmedien_gm = {
    "sz", "zeit", "faz", "tagesspiegel", "taz", "berliner", "welt", "spiegel", "stern"
}

# 3. Zielpfad für CSV definieren und ggf. alte Datei löschen
OUTPUT_CSV = "output/cluster_grossemedien.csv"
if os.path.exists(OUTPUT_CSV):
    os.remove(OUTPUT_CSV)

# 4. Verarbeitung aller relevanten CSV-Dateien im data-lake
for file in sorted(os.listdir(STORAGE_PATH)):
    if file.endswith(".csv"):
        filepath = os.path.join(STORAGE_PATH, file)
        try:
            df = pd.read_csv(filepath)

            # Filter: Nur gewünschte Medien
            df = df[df["name"].isin(zielmedien_gm)]

            # Nur verarbeiten, wenn Daten übrig sind
            if not df.empty:
                print(f"[INFO] Verarbeite Datei: {file}")
                df.apply(process_wrapper, axis=1)

                # Ergebnisse sofort speichern und Speicher leeren
                for chunk in collection:
                    chunk.to_csv(OUTPUT_CSV, mode="a", index=False, header=not os.path.exists(OUTPUT_CSV))
                collection = []

        except Exception as e:
            print(f"[ERROR] Fehler bei Datei {file}: {e}")

[INFO] Verarbeite Datei: 2021-04-01.csv
[INFO] Verarbeitung erfolgreich: sz
[INFO] Verarbeitung erfolgreich: zeit
[INFO] Verarbeitung erfolgreich: faz
[INFO] Verarbeitung erfolgreich: tagesspiegel
[INFO] Verarbeitung erfolgreich: taz
[INFO] Verarbeitung erfolgreich: berliner
[INFO] Verarbeitung erfolgreich: welt
[INFO] Verarbeitung erfolgreich: spiegel
[INFO] Verarbeitung erfolgreich: stern
[INFO] Verarbeite Datei: 2021-04-02.csv
[INFO] Verarbeitung erfolgreich: sz
[INFO] Verarbeitung erfolgreich: zeit
[INFO] Verarbeitung erfolgreich: faz
[INFO] Verarbeitung erfolgreich: tagesspiegel
[INFO] Verarbeitung erfolgreich: taz
[INFO] Verarbeitung erfolgreich: berliner
[INFO] Verarbeitung erfolgreich: welt
[INFO] Verarbeitung erfolgreich: spiegel
[INFO] Verarbeitung erfolgreich: stern
[INFO] Verarbeite Datei: 2021-04-03.csv
[INFO] Verarbeitung erfolgreich: sz
[INFO] Verarbeitung erfolgreich: zeit
[INFO] Verarbeitung erfolgreich: faz
[INFO] Verarbeitung erfolgreich: tagesspiegel
[INFO] Verarbei

In [14]:
# Pfad zur gespeicherten CSV-Datei
csv_path = "output/cluster_grossemedien.csv"

try:
    # CSV laden
    df = pd.read_csv(csv_path)

    # Anzahl Zeilen und Spalten anzeigen
    print("[INFO] Form der Datei:", df.shape)

    # Zufällige Beispielzeilen anzeigen
    print("[INFO] Beispielhafte Einträge:")
    print(df.sample(3, random_state=1))

except Exception as e:
    print("[ERROR] Fehler beim Laden der CSV-Datei:", e)

[INFO] Form der Datei: (23579891, 4)
[INFO] Beispielhafte Einträge:
          count            word        source        date
2021102       1          massen  tagesspiegel  2021-08-03
16018759      1  minderjährigen  tagesspiegel  2023-11-24
19287390      1       attentat.          welt  2024-06-07


In [17]:
# Zielmedien: regionale Medien 
zielmedien_rm = {"abendblatt", "esslinger", "suedwest", "stuttgarter", "zvw"}

In [ ]:
# 1. Import & Sammlung vorbereiten
collection = []

# 2. CSV-Datei vorbereiten
OUTPUT_CSV = "output/cluster_regiomedien.csv" 
if os.path.exists(OUTPUT_CSV):
    os.remove(OUTPUT_CSV)

# 3. Verarbeitung und direkt speichern (chunkweise!)
for file in sorted(os.listdir(STORAGE_PATH)):
    if file.endswith(".csv"):
        filepath = os.path.join(STORAGE_PATH, file)
        try:
            df = pd.read_csv(filepath)

            # Filter Zielmedien
            df = df[df["name"].isin(zielmedien_rm)] 

            if not df.empty:
                print(f"[INFO] Datei: {file}")
                df.apply(process_wrapper, axis=1)

                # Sofort speichern
                for chunk in collection:
                    chunk.to_csv(OUTPUT_CSV, mode="a", index=False, header=not os.path.exists(OUTPUT_CSV))
                collection = []

        except Exception as e:
            print(f"[ERROR] Fehler bei Datei {file}: {e}")

[INFO] Datei: 2021-04-01.csv
[INFO] Verarbeitung erfolgreich: abendblatt
[INFO] Verarbeitung erfolgreich: esslinger
[INFO] Verarbeitung erfolgreich: suedwest
[INFO] Verarbeitung erfolgreich: stuttgarter
[INFO] Verarbeitung erfolgreich: zvw
[INFO] Datei: 2021-04-02.csv
[INFO] Verarbeitung erfolgreich: abendblatt
[INFO] Verarbeitung erfolgreich: esslinger
[INFO] Verarbeitung erfolgreich: suedwest
[INFO] Verarbeitung erfolgreich: stuttgarter
[INFO] Verarbeitung erfolgreich: zvw
[INFO] Datei: 2021-04-03.csv
[INFO] Verarbeitung erfolgreich: abendblatt
[INFO] Verarbeitung erfolgreich: esslinger
[INFO] Verarbeitung erfolgreich: suedwest
[INFO] Verarbeitung erfolgreich: stuttgarter
[INFO] Verarbeitung erfolgreich: zvw
[INFO] Datei: 2021-04-04.csv
[INFO] Verarbeitung erfolgreich: abendblatt
[INFO] Verarbeitung erfolgreich: esslinger
[INFO] Verarbeitung erfolgreich: suedwest
[INFO] Verarbeitung erfolgreich: stuttgarter
[INFO] Verarbeitung erfolgreich: zvw
[INFO] Datei: 2021-04-05.csv
[INFO] Vera

In [19]:
# Pfad zur gespeicherten CSV-Datei
csv_path = "output/cluster_regiomedien.csv"

try:
    # CSV laden
    df = pd.read_csv(csv_path)

    # Anzahl Zeilen und Spalten anzeigen
    print("[INFO] Form der Datei:", df.shape)

    # Zufällige Beispielzeilen anzeigen
    print("[INFO] Beispielhafte Einträge:")
    print(df.sample(3, random_state=1))

except Exception as e:
    print("[ERROR] Fehler beim Laden der CSV-Datei:", e)

[INFO] Form der Datei: (7793586, 4)
[INFO] Beispielhafte Einträge:
         count    word       source        date
908202       1   busch    esslinger  2021-09-27
6250929      1  partei  stuttgarter  2024-06-28
1147278      1    "das          zvw  2021-11-13


In [25]:
# 1. Einzelne Cluster-Dateien laden
df_or = pd.read_csv("output/cluster_oeffentlich.csv")
df_wm = pd.read_csv("output/cluster_wirtschaft.csv")
df_gm = pd.read_csv("output/cluster_grossemedien.csv")
df_rm = pd.read_csv("output/cluster_regiomedien.csv")

In [26]:
# 2. Zusammenführen
df_medien = pd.concat([df_or, df_wm, df_gm, df_rm], axis=0, ignore_index=True)
print("[INFO] Gesamt-Daten zusammengeführt.")
print("[INFO] Zeilenanzahl:", len(df_medien))

[INFO] Gesamt-Daten zusammengeführt.
[INFO] Zeilenanzahl: 43146157


In [27]:
# 3. Speichern in SQLite-Datenbank
conn = sqlite3.connect("output/dwh.sqlite3")
df_medien.to_sql("wordcount", conn, if_exists="replace", index=False)
print("[INFO] In Datenbank gespeichert unter Tabelle 'wordcount'.")

[INFO] In Datenbank gespeichert unter Tabelle 'wordcount'.


In [28]:
# DataFrame auch als zentrale CSV speichern
df_medien.to_csv("output/df_medien.csv", index=False)
print("[INFO] Als zentrale CSV gespeichert.")

[INFO] Als zentrale CSV gespeichert.
